In [0]:
df = spark.read.format("parquet").load("abfss://bronze@azuredatabricksjesslake.dfs.core.windows.net/customers")
df.display()


In [0]:
df = df.drop("_rescued_data")

from pyspark.sql.functions import *

In [0]:
df = df.withColumn("domain", split(col("email"),"@")[1])
df.display()

In [0]:

df.groupBy("domain").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()


In [0]:
df = df.withColumn("full_name", concat("first_name", lit(" "), "last_name"))
df = df.drop("first_name", "last_name")
df.display()

In [0]:
df.write.format("delta").mode("append").option("path", "abfss://silver@azuredatabricksjesslake.dfs.core.windows.net/customers").saveAsTable("dtbproject.silver.customers")

When trying to validated all my data before I start kickstarting the curated dataset I just felt the need to change the position of the "fullname" column

In [0]:
df = spark.read.format("delta").load("abfss://silver@azuredatabricksjesslake.dfs.core.windows.net/customers")
df.display()

In [0]:
df = df.select("customer_id", "full_name", "email", "city", "state", "domain")
df.write.format("delta").mode("overwrite").option("path", "abfss://silver@azuredatabricksjesslake.dfs.core.windows.net/customers").saveAsTable("dtbproject.silver.customers")

In [0]:
df = spark.read.table("dtbproject.silver.customers")

In [0]:
df.display()

In [0]:
df_check = spark.sql(
    '''
    SELECT * FROM dtbproject.silver.customers
    '''
) 

df_check.display()